In [ ]:
pip install torchvision

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
#mount drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from torchvision import models
import torch
import os
import time

import json
import numpy as np
from torchvision import transforms
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torchvision.datasets import CIFAR10
import random
from PIL import Image

#dir(models)

In [ ]:
#fix random seed
SEED = 42

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

In [ ]:
len(dataset_train)

576

In [ ]:
efficientNet = models.efficientnet_b0(pretrained=True)

/usr/local/lib/python3.7/dist-packages/torchvision/models/_utils.py:209: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  f"The parameter '{pretrained_param}' is deprecated since 0.13 and will be removed in 0.15, "
/usr/local/lib/python3.7/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=EfficientNet_B0_Weights.IMAGENET1K_V1`. You can also use `weights=EfficientNet_B0_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/efficientnet_b0_rwightman-3dd342df.pth" to /root/.cache/torch/hub/checkpoints/efficientnet_b0_rwightman-3dd342df.pth


  0%|          | 0.00/20.5M [00:00<?, ?B/s]

In [ ]:
efficientNet.eval()

EfficientNet(
  (features): Sequential(
    (0): Conv2dNormActivation(
      (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): SiLU(inplace=True)
    )
    (1): Sequential(
      (0): MBConv(
        (block): Sequential(
          (0): Conv2dNormActivation(
            (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
            (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (2): SiLU(inplace=True)
          )
          (1): SqueezeExcitation(
            (avgpool): AdaptiveAvgPool2d(output_size=1)
            (fc1): Conv2d(32, 8, kernel_size=(1, 1), stride=(1, 1))
            (fc2): Conv2d(8, 32, kernel_size=(1, 1), stride=(1, 1))
            (activation): SiLU(inplace=True)
            (scale_activation): Sigmoid()
          )
          (2): Conv2dNormActivat

In [ ]:
efficientNet.classifier[1] = nn.Linear(1280, 2) #change output fc layer (to obtain the number of classes desired)

In [ ]:
efficientNet.eval()

EfficientNet(
  (features): Sequential(
    (0): Conv2dNormActivation(
      (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): SiLU(inplace=True)
    )
    (1): Sequential(
      (0): MBConv(
        (block): Sequential(
          (0): Conv2dNormActivation(
            (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
            (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (2): SiLU(inplace=True)
          )
          (1): SqueezeExcitation(
            (avgpool): AdaptiveAvgPool2d(output_size=1)
            (fc1): Conv2d(32, 8, kernel_size=(1, 1), stride=(1, 1))
            (fc2): Conv2d(8, 32, kernel_size=(1, 1), stride=(1, 1))
            (activation): SiLU(inplace=True)
            (scale_activation): Sigmoid()
          )
          (2): Conv2dNormActivat

In [ ]:
#loading the saved file once again

dataset_train = torch.load('/content/drive/MyDrive/Vision/Human_dataset/train_all.pt')

dataset_valid = torch.load('/content/drive/MyDrive/Vision/Human_dataset/valid_all.pt')

dataset_test = torch.load('/content/drive/MyDrive/Vision/Human_dataset/test_all.pt')

In [ ]:
#list to encode/decode labels
indices_to_labels = ['human', 'statue']
labels_to_indices = {'human': 0, 'statue': 1}

In [ ]:
#create labels

train_labels = []
for i in range(0,500):
  train_labels.append('human')
for i in range(0,500):
  train_labels.append('statue')

valid_labels = []
test_labels = []

for i in range(0,55):
  valid_labels.append('human')
  test_labels.append('human')
for i in range(0,55):
  valid_labels.append('statue')
  test_labels.append('statue')

In [ ]:
#encode/decode labels functions

def encode_labels(labels, labels_to_indices):
  encoded = []
  for lab in labels:
    encoded.append(labels_to_indices[lab])
  return encoded

def decode_labels(encoded, indices_to_labels):
  labels = []
  for enc in encoded:
    labels.append(indices_to_labels[enc])
  return labels

def create_hot_encoder(encoded, num_classes):
  hot_encoder = torch.zeros(len(encoded), num_classes)
  c = 0
  for i in encoded:
    hot_encoder[c][i] = 1
    c += 1
  return hot_encoder


In [ ]:
train_labels = encode_labels(train_labels, labels_to_indices)
valid_labels = encode_labels(valid_labels, labels_to_indices)
test_labels = encode_labels(test_labels, labels_to_indices)

In [ ]:
hot_encoder_train = create_hot_encoder(train_labels, 2)
hot_encoder_valid = create_hot_encoder(valid_labels, 2)
hot_encoder_test = create_hot_encoder(test_labels, 2)

In [ ]:
len(hot_encoder_train)

1000

In [ ]:
data_train = {}
for i in range(0, len(dataset_train)):
  data_train[i]={'inputs': dataset_train[i], 'outputs': hot_encoder_train[i]}
  #data[i]={'inputs': torch.LongTensor(dataset[i]), 'outputs': hot_encoder[i]}

data_valid = {}
for i in range(0, len(dataset_valid)):
  data_valid[i]={'inputs': dataset_valid[i], 'outputs': hot_encoder_valid[i]}

data_test = {}
for i in range(0, len(dataset_test)):
  data_test[i]={'inputs': dataset_test[i], 'outputs': hot_encoder_test[i]}

In [ ]:
train_dataloader = DataLoader(data_train, batch_size=8, shuffle=True)
valid_dataloader = DataLoader(data_valid, batch_size=8, shuffle=False)
test_dataloader = DataLoader(data_test, batch_size=8, shuffle=False)

In [ ]:
def train_model(model, train_dataloader, criterion, optimizer, valid_dataloader=None, num_epochs=25, is_inception=False):
    since = time.time()
    since2 = since

    val_acc_history = []

    #best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('----------')
        
        model.train()  # Set model to training mode
        running_loss = 0.0
        running_corrects = 0

        # Iterate over data.
        for data in train_dataloader:
          inputs = torch.squeeze(data['inputs'])
          labels = data['outputs']
          #inputs = inputs.to(device)
          #labels = labels.to(device)

          # zero the parameter gradients
          optimizer.zero_grad()

          # forward
          # track history if only in train
          #with torch.set_grad_enabled(True):
            # Get model outputs and calculate loss
            # Special case for inception because in training it has an auxiliary output. In train
            #   mode we calculate the loss by summing the final output and the auxiliary output
            #   but in testing we only consider the final output.
          '''if is_inception:
              # From https://discuss.pytorch.org/t/how-to-optimize-inception-model-with-auxiliary-classifiers/7958
              outputs, aux_outputs = model(inputs)
              loss1 = criterion(outputs, labels)
              loss2 = criterion(aux_outputs, labels)
              loss = loss1 + 0.4*loss2'''
            #else:

          outputs = model(inputs)
          loss = criterion(outputs, labels)

          _, preds = torch.max(outputs, 1)

            # backward + optimize only if in training phase
          loss.backward()
          optimizer.step()

            # statistics
          running_loss += loss.item() * inputs.size(0)
          _, correct = torch.max(labels, 1)
          running_corrects += torch.sum(preds == correct)

          epoch_loss = running_loss / len(train_dataloader.dataset)
            #epoch_acc = running_corrects.double() / len(train_dataloader.dataset)

          #print('Loss: {:.4f}'.format(epoch_loss))
            #print('{} Loss: {:.4f} Acc: {:.4f}'.format(epoch_loss, epoch_acc))

          '''
            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())
            if phase == 'val':
                val_acc_history.append(epoch_acc)
          '''

        print()
        print('Total Epoch loss: ', epoch_loss)
        print('Total correct classified: ', running_corrects, ' total classified: ', len(train_dataloader)*8)
        time_elapsed = time.time() - since2
        since2 = time.time()
        print('Time to complete epochs {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))

        if valid_dataloader:
          print('Validation Results:')
          evaluate_model (model, valid_dataloader)

        running_loss = 0.0
        running_corrects = 0

    print('-----end-----')
    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    #print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    #model.load_state_dict(best_model_wts)
    return model



def evaluate_model(model, dataloaders, top_three=False):
  running_corrects = 0
  for data in dataloaders:
    inputs = torch.squeeze(data['inputs'])
    labels = data['outputs']
    outputs = model(inputs)

    _, index = torch.max(outputs, 1)

    percentage = torch.nn.functional.softmax(outputs, dim=1)[0] * 100

    #print('The class assigned is: ', index[0], ' with the percentage: ', percentage[index[0]].item())

    if top_three:
      _, indices = torch.sort(outputs, descending=True)
      [(idx, percentage[idx].item()) for idx in indices[0][:3]]
      print(index[0], percentage[index[0]].item())

    _, correct = torch.max(labels, 1)
    running_corrects += torch.sum(index == correct)

        
  print('Total correct classified: ', running_corrects, ' total classified: ', len(dataloaders)*8)


In [ ]:
loss_function = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(efficientNet.parameters())

In [ ]:
efficientNet = train_model(efficientNet, train_dataloader, loss_function, optimizer, valid_dataloader, 5)

Epoch 0/4
----------

Total Epoch loss:  0.09439919372065925
Total correct classified:  tensor(972)  total classified:  1000
Time to complete epochs 5m 7s
Validation Results:
Total correct classified:  tensor(74)  total classified:  112
Epoch 1/4
----------

Total Epoch loss:  0.07533677394582264
Total correct classified:  tensor(979)  total classified:  1000
Time to complete epochs 5m 7s
Validation Results:
Total correct classified:  tensor(62)  total classified:  112
Epoch 2/4
----------

Total Epoch loss:  0.10981000323523768
Total correct classified:  tensor(963)  total classified:  1000
Time to complete epochs 5m 5s
Validation Results:
Total correct classified:  tensor(67)  total classified:  112
Epoch 3/4
----------

Total Epoch loss:  0.012292286497104215
Total correct classified:  tensor(995)  total classified:  1000
Time to complete epochs 5m 6s
Validation Results:
Total correct classified:  tensor(67)  total classified:  112
Epoch 4/4
----------

Total Epoch loss:  0.00554524

In [ ]:
#save
PATH2 = '/content/drive/MyDrive/Vision/Human_dataset/EfficientNet_back.pt'
torch.save(efficientNet, PATH2)

In [ ]:
#load
PATH2 = '/content/drive/MyDrive/Vision/Human_dataset/EfficientNet_back.pt'
model = torch.load(PATH2)
model.eval()

EfficientNet(
  (features): Sequential(
    (0): Conv2dNormActivation(
      (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): SiLU(inplace=True)
    )
    (1): Sequential(
      (0): MBConv(
        (block): Sequential(
          (0): Conv2dNormActivation(
            (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
            (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (2): SiLU(inplace=True)
          )
          (1): SqueezeExcitation(
            (avgpool): AdaptiveAvgPool2d(output_size=1)
            (fc1): Conv2d(32, 8, kernel_size=(1, 1), stride=(1, 1))
            (fc2): Conv2d(8, 32, kernel_size=(1, 1), stride=(1, 1))
            (activation): SiLU(inplace=True)
            (scale_activation): Sigmoid()
          )
          (2): Conv2dNormActivat

In [ ]:
evaluate_model(model, test_dataloader)

Total correct classified:  tensor(108)  total classified:  112


In [ ]:
from PIL import Image

transform = transforms.Compose([transforms.Resize(256), transforms.CenterCrop(256), transforms.ToTensor()])

#response = requests.get('/content/drive/MyDrive/Vision/Try/a.jpg')
img = Image.open('/content/drive/MyDrive/Vision/Try/IMG_9863.jpg')
img_t = transform(img)
batch_t = torch.unsqueeze(img_t, 0)
out = model(batch_t)
print(out.shape)

torch.Size([1, 2])


In [ ]:
_, index = torch.max(out, 1)
percentage = torch.nn.functional.softmax(out, dim=1)[0] * 100
print(index[0], percentage[index[0]].item())

tensor(1) 100.0


In [ ]:
_, indices = torch.sort(out, descending=True)
[(idx, percentage[idx].item()) for idx in indices[0][:5]]

[(tensor(1), 100.0), (tensor(0), 4.634537048589148e-18)]

In [ ]:
index[0].item()

1

# Python face detector



In [ ]:
pip install facenet-pytorch

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.9 MB 23.8 MB/s 


In [ ]:
import cv2
import numpy as np
import torch
import argparse
#import utils
from PIL import Image
from facenet_pytorch import MTCNN

In [ ]:
#mount drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# create the argument parser
parser = argparse.ArgumentParser()
parser.add_argument('-i', '--input', required=True, help='path to the input data')
args = vars(parser.parse_args())

In [ ]:
# computation device

#device = torch.device('cuda' if torch.cuda.is_available else 'cpu')
#device = torch.divice('cpu')

# create the MTCNN model, `keep_all=True` returns all the detected faces 
mtcnn = MTCNN(keep_all=True, device='cpu')

In [ ]:
# read the image 

#image = Image.open(args['input']).convert('RGB')
image = Image.open('/content/drive/MyDrive/Vision/Try/IMG_9851.jpg').convert('RGB')

# create an image array copy so that we can use OpenCV functions on it
image_array = np.array(image, dtype=np.float32)
# cv2 image color conversion
image_array = cv2.cvtColor(image_array, cv2.COLOR_RGB2BGR)

In [ ]:
# the detection module returns the bounding box coordinates and confidence ...
# ... by default, to get the facial landmarks, we have to provide ...
# ... `landmarks=True`
bounding_boxes, conf, landmarks = mtcnn.detect(image, landmarks=True)
# print(f"Bounding boxes shape: {bounding_boxes.shape}")
# print(f"Landmarks shape: {landmarks.shape}")

In [ ]:
#print(bounding_boxes)
#print('len: ', len(bounding_boxes))
print(conf[0])
print('len: ', len(conf))
#print(landmarks)
#print('len: ', len(landmarks))

0.99859315
len:  1


In [ ]:
image = Image.open('/content/drive/MyDrive/Vision/Try/00001.jpg')
print(check_faces(image))

False


In [ ]:
#check if there is a face
#(return 'False' if there isn't any face or if there is more than one face)

def check_faces(img):
  image = img.convert('RGB')
  image_array = np.array(image, dtype=np.float32)
  # cv2 image color conversion
  image_array = cv2.cvtColor(image_array, cv2.COLOR_RGB2BGR)
  bounding_boxes, conf = mtcnn.detect(image)

  if conf[0] == None:
    return False
  else:
    if len(conf) == 1 and conf[0] > 0.59:
      return True
    else:
      return False